In [1]:
%matplotlib widget

In [2]:
from glob import glob
import numpy as np
import pandas as pd
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment
from fimpy.pipeline.general import calc_f0, dff
from motions.utilities import stim_vel_dir_dataframe, quantize_directions
from scipy.interpolate import interp1d 
from scipy.signal import convolve2d
import colorspacious
import matplotlib.pyplot as plt
import json
from pathlib import Path

In [170]:
master = Path(r"Z:\Hagar\E0040\v31\post ablation 2d")
fish_list = list(master.glob("*_f*"))
path = fish_list[0]
print(path)


Z:\Hagar\E0040\v31\post ablation 2d\230426_f0_post


In [171]:
exp = glob(str(path / "*behavior*"))[0]
traces = fl.load(path / "filtered_traces.h5", "/undetr")

In [172]:
coords = fl.load(path / "data_from_suite2p_cells.h5", "/coords")
suite2p_brain = fl.load(path / "data_from_suite2p_cells_brain.h5")
in_brain_idx = suite2p_brain['coords_idx']

In [173]:
with open(next(Path(path).glob("*metadata.json")), "r") as f:
        metadata = json.load(f)
lsconfig = metadata["imaging"]["microscope_config"]['lightsheet']['scanning']
z_tot_span = lsconfig["z"]["piezo_max"] - lsconfig["z"]["piezo_min"]
n_planes = lsconfig["triggering"]["n_planes"]
z_res = z_tot_span / n_planes
res = [0.6, 0.6, z_res]

In [174]:
sensory_regressors = fl.load(path / "sensory_regressors.h5", "/regressors")
titles = ['right', 'backward right', 'backward', 'backward left', 'left', 'forward left', 'forward', 'forward right', ]

In [175]:
sensory_regressors

,motion_0,motion_1,motion_2,motion_3,motion_4,motion_5,motion_6,motion_7
0,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...
3875,0.969769,2.037189e-52,2.734777e-12,2.588547e-20,2.842649e-08,2.462895e-28,2.161375e-44,1.906141e-60
3876,0.976006,1.616918e-52,2.170594e-12,2.054531e-20,2.256212e-08,1.954801e-28,1.715484e-44,1.512905e-60
3877,0.980956,1.283349e-52,1.722802e-12,1.630682e-20,1.790756e-08,1.551526e-28,1.361581e-44,1.200793e-60
3878,0.984885,1.018594e-52,1.367389e-12,1.294273e-20,1.421324e-08,1.231447e-28,1.080687e-44,9.530703e-61


In [176]:
fig, axs = plt.subplots(2, 8, figsize=(15, 5), gridspec_kw={'width_ratios': [6, 2, 6, 2, 6, 2, 6, 2], 'height_ratios': [1, 3]})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [177]:
for direction in range(0, 8, 2):
    print(direction)
    #traces = traces[:1600]
    current_dir = np.asarray(sensory_regressors.iloc[:, direction])
    #reg_values = current_dir @ traces[:, :] 
    num_traces = np.shape(traces)[1]
        
    reg_corr = np.zeros((num_traces))
    for i in range(num_traces):
        reg_corr[i] = np.corrcoef(current_dir, traces[:, i])[0,1]
       
    coords_ib = coords[in_brain_idx]
    reg_corr_ib = reg_corr[in_brain_idx]
        
    mp_ind = np.argsort(np.abs(reg_corr_ib))
    
    axs[1,direction].scatter(coords_ib[mp_ind,2]*res[0], coords_ib[mp_ind,1]*res[1], c=reg_corr_ib[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
    axs[1,direction+1].scatter(coords_ib[mp_ind,0]*res[2], coords_ib[mp_ind,1]*res[1], c=reg_corr_ib[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
    axs[0,direction].scatter(coords_ib[mp_ind,2]*res[0], coords_ib[mp_ind,0]*res[2], c=reg_corr_ib[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)

    axs[0,direction].spines['right'].set_visible(False)
    axs[0,direction].spines['top'].set_visible(False)

    axs[1,direction+1].spines['right'].set_visible(False)
    axs[1,direction+1].spines['top'].set_visible(False)

    axs[1,direction].spines['right'].set_visible(False)
    axs[1,direction].spines['top'].set_visible(False)

    axs[0,direction+1].axis('off')

    axs[0,direction].set_title(titles[direction])


0
2
4
6


In [178]:
fig.subplots_adjust(left=0.05, wspace=0.3)

In [179]:
file_name = "corr map all directions.jpg"
fig.savefig(path / file_name, dpi=300)

In [180]:
np.shape(current_dir)

(3880,)

In [ ]:
reg_values = current_dir @ traces[:, :] 

In [ ]:
num_traces = np.shape(traces)[1]
print(num_traces)
reg_corr = np.zeros((num_traces))
for i in range(num_traces):
    reg_corr[i] = np.corrcoef(current_dir, traces[:, i])[0,1]

In [ ]:
coords_ib = coords[in_brain_idx]
reg_values_ib = reg_values[in_brain_idx]
reg_corr_ib = reg_corr[in_brain_idx]

In [ ]:
titles = ['right', 'backward right', 'backward', 'backward left', 'left', 'forward left', 'forward', 'forward right', ]

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(6, 6), gridspec_kw={'width_ratios': [6, 2], 'height_ratios': [1, 2]})
mp_ind = np.argsort(np.abs(reg_corr_ib))
axs[1,0].scatter(coords_ib[mp_ind,2]*0.6, coords_ib[mp_ind,1]*.6, c=reg_corr_ib[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
axs[1,1].scatter(coords_ib[mp_ind,0]*z_res, coords_ib[mp_ind,1]*0.6, c=reg_corr_ib[mp_ind], alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
axs[0,0].scatter(coords_ib[mp_ind,2]*0.6, coords_ib[mp_ind,0]*z_res, c=reg_corr_ib[mp_ind], alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)

axs[0,0].spines['right'].set_visible(False)
axs[0,0].spines['top'].set_visible(False)

axs[1,1].spines['right'].set_visible(False)
axs[1,1].spines['top'].set_visible(False)

axs[1,0].spines['right'].set_visible(False)
axs[1,0].spines['top'].set_visible(False)

axs[0,1].axis('off')

fig.suptitle(titles[direction])

In [ ]:
file_name = "corr map " + titles[direction] + ".jpg"
fig.savefig(path / file_name, dpi=300)